In [1]:
from tqdm import tqdm

import numpy as np

import sgkit as sg
from sgkit.io.vcf import vcf_to_zarr

import tszip

import sys
sys.path.append("../../tsimpute/src/")
import compare_vcfs as cv
import impute_by_sample_matching as impute
import util


In [2]:
trees_dir = "../data/trees/"
zarr_dir = "../data/zarr/"
vcf_dir = "../data/vcf/"
lshmm_dir = "../analysis/lshmm/"


In [3]:
# Ref. panel containing sites that are biallelic in the ref. samples
ref_ts_file = trees_dir + "ref.tsz"
ref_ts = tszip.decompress(ref_ts_file)
ref_ts


In [4]:
# Genetic variants in target cohort at chip-like sites
target_ds = sg.load_dataset(zarr_dir + "target.zarr")   # For VCF header only
target_sample_names = target_ds.vcf_header.split("\n")[5].split("\t")[9:]

target_chip_ds_compat = sg.load_dataset(zarr_dir + "target_chip_compat.zarr")   # ACGT space
target_chip_ds_compat


<xarray.Dataset>
Dimensions:             (variants: 7899, samples: 176, ploidy: 2, contigs: 1,
                         alleles: 4)
Dimensions without coordinates: variants, samples, ploidy, contigs, alleles
Data variables:
    call_genotype       (variants, samples, ploidy) float64 dask.array<chunksize=(1975, 44, 1), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(3950, 88, 1), meta=np.ndarray>
    contig_id           (contigs) <U2 dask.array<chunksize=(1,), meta=np.ndarray>
    sample_id           (samples) object dask.array<chunksize=(176,), meta=np.ndarray>
    variant_allele      (variants, alleles) <U1 dask.array<chunksize=(7899, 4), meta=np.ndarray>
    variant_contig      (variants) int8 dask.array<chunksize=(7899,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(7899,), meta=np.ndarray>
Attributes:
    contigs:  ['20']
    source:   sgkit-0.7.0

In [5]:
# Reduce the ref. ts to only the chip-like sites
ref_chip_ts = ref_ts.delete_sites(
    site_ids=np.arange(ref_ts.num_sites)[
        np.isin(
            ref_ts.sites_position,
            target_chip_ds_compat.variant_position.values,
            invert=True,
        )
    ]
)


In [7]:
print(f"Sites in ref. ts (chip-like sites): {ref_chip_ts.num_sites}")
print(f"Sites in target ds (chip-like sites): {len(target_chip_ds_compat.variant_position)}")
print(f"Are they identical? {np.array_equal(ref_chip_ts.sites_position, target_chip_ds_compat.variant_position.values)}")


Sites in ref. ts (chip-like sites): 7899
Sites in target ds (chip-like sites): 7899
Are they identical? True


In [ ]:
def wrapper_add_individual_to_ts(ts, sample_paths, sample_names, precision):
    new_ts = ts

    i = 0
    for sample_name in tqdm(sample_names):
        metadata_str = f"\"name\": \"{sample_name}\", "
        metadata_str += f"\"status\": \"imputed\", "
        metadata_str += f"\"recomb\": \"uniform\", "
        metadata_str += f"\"precision\": \"{precision}\""

        path_1 = util.SamplePath(
            individual=sample_name,
            nodes=sample_paths[2 * i],
            site_positions=ts.sites_position,
            metadata=metadata_str.encode('ascii')
        )
        path_2 = util.SamplePath(
            individual=sample_name,
            nodes=sample_paths[2 * i + 1],
            site_positions=ts.sites_position,
            metadata=metadata_str.encode('ascii')
        )

        assert path_1.is_valid
        assert path_2.is_valid

        _, new_ts = util.add_individual_to_tree_sequence(
            ts=new_ts,
            paths=[path_1, path_2],
            metadata=metadata_str.encode('ascii')
        )

        i += 1

    return new_ts


### Run lshmm under different parameters


In [19]:
# Run lshmm
## switch prob.   = 1e-08
## mismatch prob. = 1e-08

In [9]:
out_prefix = "target_lshmm_chip_r08_m08"
switch_prob = np.repeat(1e-08, ref_chip_ts.num_sites)
mismatch_prob = np.repeat(1e-08, ref_chip_ts.num_sites)
precision = 25


In [10]:
_, h2_r08_m08, _ = impute.impute_by_sample_matching(
    ref_ts=ref_chip_ts,
    target_ds=target_chip_ds_compat,
    switch_prob=switch_prob,
    mismatch_prob=mismatch_prob,
    precision=precision,
)


100%|██████████| 7899/7899 [00:00<00:00, 19315.51it/s]


In [11]:
assert h2_r08_m08.shape[0] == target_chip_ds_compat.dims["samples"] * target_chip_ds_compat.dims["ploidy"]
assert h2_r08_m08.shape[1] == ref_chip_ts.num_sites


In [13]:
lshmm_chip_ts_r08_m08 = wrapper_add_individual_to_ts(
    ts=ref_chip_ts,
    sample_paths=h2_r08_m08,
    sample_names=target_sample_names,
    precision=precision,
)


100%|██████████| 176/176 [03:37<00:00,  1.23s/it]


In [14]:
lshmm_chip_ts_r08_m08_file = lshmm_dir + "target_lshmm_chip_r08_m08.tsz"
tszip.compress(lshmm_chip_ts_r08_m08, lshmm_chip_ts_r08_m08_file)


In [16]:
lshmm_chip_vcf_file = vcf_dir + "target_lshmm_chip_r08_m08.vcf"
with open(lshmm_chip_vcf_file, "w") as f:
    lshmm_chip_ts_r08_m08.write_vcf(
        output=f,
        contig_id="20",
        individuals=np.arange(
            ref_chip_ts.num_individuals,
            lshmm_chip_ts_r08_m08.num_individuals
        ),
        individual_names=target_sample_names,
    )


In [17]:
%%bash
vcf_dir="../data/vcf/"
bgzip ${vcf_dir}"target_lshmm_chip_r08_m08.vcf"
bcftools index ${vcf_dir}"target_lshmm_chip_r08_m08.vcf.gz"


In [18]:
lshmm_chip_vcf_file = vcf_dir + out_prefix + ".vcf.gz"
vcf_to_zarr(lshmm_chip_vcf_file, zarr_dir + out_prefix + ".zarr")
lshmm_chip_r08_m08_ds = sg.load_dataset(zarr_dir + out_prefix + ".zarr")
lshmm_chip_r08_m08_ds_compat = cv.remap_to_acgt(lshmm_chip_r08_m08_ds, num_workers=6)


100%|██████████| 7899/7899 [00:37<00:00, 211.57it/s]


In [20]:
# Run lshmm
## switch prob.   = 1e-08
## mismatch prob. = 1e-20

In [21]:
out_prefix = "target_lshmm_chip_r08_m20"
switch_prob = np.repeat(1e-08, ref_chip_ts.num_sites)
mismatch_prob = np.repeat(1e-20, ref_chip_ts.num_sites)
precision = 25


In [22]:
_, h2_r08_m20, _ = impute.impute_by_sample_matching(
    ref_ts=ref_chip_ts,
    target_ds=target_chip_ds_compat,
    switch_prob=switch_prob,
    mismatch_prob=mismatch_prob,
    precision=precision,
)


100%|██████████| 7899/7899 [00:00<00:00, 17616.68it/s]


In [23]:
assert h2_r08_m20.shape[0] == target_chip_ds_compat.dims["samples"] * target_chip_ds_compat.dims["ploidy"]
assert h2_r08_m20.shape[1] == ref_chip_ts.num_sites


In [24]:
lshmm_chip_ts_r08_m20 = wrapper_add_individual_to_ts(
    ts=ref_chip_ts,
    sample_paths=h2_r08_m20,
    sample_names=target_sample_names,
    precision=precision,
)


100%|██████████| 176/176 [03:35<00:00,  1.22s/it]


In [25]:
lshmm_chip_ts_r08_m20_file = lshmm_dir + out_prefix + ".tsz"
tszip.compress(lshmm_chip_ts_r08_m20, lshmm_chip_ts_r08_m20_file)


In [26]:
lshmm_chip_vcf_file = vcf_dir + out_prefix + ".vcf"
with open(lshmm_chip_vcf_file, "w") as f:
    lshmm_chip_ts_r08_m20.write_vcf(
        output=f,
        contig_id="20",
        individuals=np.arange(
            ref_chip_ts.num_individuals,
            lshmm_chip_ts_r08_m20.num_individuals
        ),
        individual_names=target_sample_names,
    )


In [27]:
%%bash
vcf_dir="../data/vcf/"
bgzip ${vcf_dir}"target_lshmm_chip_r08_m20.vcf"
bcftools index ${vcf_dir}"target_lshmm_chip_r08_m20.vcf.gz"


In [28]:
lshmm_chip_vcf_file = vcf_dir + out_prefix + ".vcf.gz"
vcf_to_zarr(lshmm_chip_vcf_file, zarr_dir + out_prefix + ".zarr")
lshmm_chip_r08_m20_ds = sg.load_dataset(zarr_dir + out_prefix + ".zarr")
lshmm_chip_r08_m20_ds_compat = cv.remap_to_acgt(lshmm_chip_r08_m20_ds, num_workers=6)


100%|██████████| 7899/7899 [00:37<00:00, 208.62it/s]


In [29]:
# Run lshmm
## switch prob.   = 1e-06
## mismatch prob. = 1e-06

In [30]:
out_prefix = "target_lshmm_chip_r06_m06"
switch_prob = np.repeat(1e-06, ref_chip_ts.num_sites)
mismatch_prob = np.repeat(1e-06, ref_chip_ts.num_sites)
precision = 25


In [31]:
_, h2_r06_m06, _ = impute.impute_by_sample_matching(
    ref_ts=ref_chip_ts,
    target_ds=target_chip_ds_compat,
    switch_prob=switch_prob,
    mismatch_prob=mismatch_prob,
    precision=precision,
)


100%|██████████| 7899/7899 [00:00<00:00, 17813.12it/s]


In [32]:
assert h2_r06_m06.shape[0] == target_chip_ds_compat.dims["samples"] * target_chip_ds_compat.dims["ploidy"]
assert h2_r06_m06.shape[1] == ref_chip_ts.num_sites


In [33]:
lshmm_chip_ts_r06_m06 = wrapper_add_individual_to_ts(
    ts=ref_chip_ts,
    sample_paths=h2_r06_m06,
    sample_names=target_sample_names,
    precision=precision,
)


100%|██████████| 176/176 [03:42<00:00,  1.26s/it]


In [34]:
out_prefix = "target_lshmm_chip_r06_m06"
lshmm_chip_ts_r06_m06_file = lshmm_dir + out_prefix + ".tsz"
tszip.compress(lshmm_chip_ts_r06_m06, lshmm_chip_ts_r06_m06_file)


In [35]:
lshmm_chip_vcf_file = vcf_dir + out_prefix + ".vcf"
with open(lshmm_chip_vcf_file, "w") as f:
    lshmm_chip_ts_r06_m06.write_vcf(
        output=f,
        contig_id="20",
        individuals=np.arange(
            ref_chip_ts.num_individuals,
            lshmm_chip_ts_r06_m06.num_individuals
        ),
        individual_names=target_sample_names,
    )


In [36]:
%%bash
vcf_dir="../data/vcf/"
bgzip ${vcf_dir}"target_lshmm_chip_r06_m06.vcf"
bcftools index ${vcf_dir}"target_lshmm_chip_r06_m06.vcf.gz"


In [37]:
lshmm_chip_vcf_file = vcf_dir + out_prefix + ".vcf.gz"
vcf_to_zarr(lshmm_chip_vcf_file, zarr_dir + out_prefix + ".zarr")
lshmm_chip_r06_m06_ds = sg.load_dataset(zarr_dir + out_prefix + ".zarr")
lshmm_chip_r06_m06_ds_compat = cv.remap_to_acgt(lshmm_chip_r06_m06_ds, num_workers=6)


100%|██████████| 7899/7899 [00:38<00:00, 205.53it/s]


### Compare imputed genotypes from different lshmm runs


In [38]:
def compare_genotypes(ds1, ds2):
    assert np.array_equal(
        ds1.variant_position,
        ds2.variant_position,
    )

    num_sites_different = 0
    for i in np.arange(ds1.dims['variants']):
        if not np.array_equal(
            ds1.call_genotype[i].values,
            ds2.call_genotype[i].values
        ):
            #print(ds1.variant_position.values[i])
            num_sites_different += 1

    return num_sites_different


In [43]:
compare_genotypes(lshmm_chip_r08_m08_ds_compat, target_chip_ds_compat)


3173

In [44]:
compare_genotypes(lshmm_chip_r08_m20_ds_compat, target_chip_ds_compat)


283

In [45]:
compare_genotypes(lshmm_chip_r06_m06_ds_compat, target_chip_ds_compat)


3347